In [8]:
import time
tstart=time.clock()
print(tstart)
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import pandas as pd
import pqkmeans

In [5]:
def PQKMeansGen(raster, output, k):
        NDBI = np.ma.masked_values(rasterio.open(raster).read(1),rasterio.open(raster).nodata)
        MEI = np.ma.masked_values(rasterio.open(raster).read(2),rasterio.open(raster).nodata)
        VIGS = np.ma.masked_values(rasterio.open(raster).read(3),rasterio.open(raster).nodata)
        DBI = np.ma.masked_values(rasterio.open(raster).read(4),rasterio.open(raster).nodata)
        Qz_Calcite = np.ma.masked_values(rasterio.open(raster).read(5),rasterio.open(raster).nodata)
        VV = np.ma.masked_values(rasterio.open(raster).read(6),rasterio.open(raster).nodata)
        VH = np.ma.masked_values(rasterio.open(raster).read(7),rasterio.open(raster).nodata)
        
        ###Based on previous failures, the author recommends to use arrays instead of data from pandas in order to have (input values, ndimension):
        
        data = pd.DataFrame({"NDBI":NDBI[NDBI==NDBI],"MEI":MEI[MEI==MEI], "VIGS":VIGS[VIGS==VIGS], "DBI":DBI[DBI==DBI], "Qz_Calcite":Qz_Calcite[Qz_Calcite==Qz_Calcite],"VV":VV[VV==VV], "VH":VH[VH==VH]})
        #for calculations using algorithms we have to drop NaN values. previous step!!!!
        data2= data.dropna()
        
        ####but pandas when read in the following form also works:
        X = data2[["NDBI", "MEI", "VIGS","DBI", "Qz_Calcite", "VV", "VH"]]

        ####but has some limitations:
        
        #####so.....
        X = np.asarray((data2[["NDBI", "MEI", "VIGS","DBI", "Qz_Calcite", "VV", "VH"]]))
        print(X.nbytes)
        print(X.shape)
        ####Train the encoder!!!
        ####Because we have 12 bands (12D input), our num_subdim or M has to be multiple of the input dimension:
        encoder = pqkmeans.encoder.PQEncoder(num_subdim=7, Ks=256)
        encoder.fit(X[:5000000])
        X_pqcode = encoder.transform(X)
        print(X_pqcode.shape)
        tmid=time.clock()
        print(tmid)
      
        
        ####As can be seen, the reconstructed vectors are similar to the original one.
        ###It allows you to compress input vectors to PQ-codes, and store the PQ-codes only (X2_pqcode) In a large-scale data processing scenario
        np.save("pqcode.npy", X_pqcode)

        ####Clustering
        pqkmean = pqkmeans.clustering.PQKMeans(encoder=encoder, k=k)
        Labels = pqkmean.fit_predict(X_pqcode) 
        
        #the array of the KMeans result does not contain the NaN values, so it is impossible to reshape to its original shape(raster). the nextstep is to find a way to add the Nan values (then nodata) to the labels. Maybe a for loop!!!
        Z= pd.DataFrame({"Labels":Labels})
        Z_Reindexed = Z.set_index(data2.index)
        data["Label"] = Z_Reindexed["Labels"]
        Result = pd.to_numeric(data["Label"], downcast = "float" )
        im = Result.values
        im = np.reshape(Result.values, (NDBI.shape[0], NDBI.shape[1] ))
        ##### After, Save
        PQKMean_output = rasterio.open(output, "w", driver = "GTiff", height = NDBI.shape[0], width = NDBI.shape[1], dtype = NDBI.dtype, count = 1, nodata = rasterio.open(raster).nodata, crs = "+proj = utm", transform = rasterio.open(raster).transform)
        PQKMean_output.write(im, 1)
        PQKMean_output.close()


### Run the code below, and open the image from the output folder in arcGIS Pro (arcmap does not open properly (32bit!!))

In [7]:
raster= "/Users/arngolo/Downloads/RESEARCH_Angolo/PYTHON/NDBI_MEI_VIGS_DBI_QzCa_VV_VH_10m.tif"
output="/Users/arngolo/Downloads/RESEARCH_Angolo/PYTHON/Program_Result/L8_PqKm_NDBI_MEI_VIGS_DBI_Qz_Calcite_VV_VH_K8_8bitsAAAAAAA.tif"
k=8
PQKMeansGen(raster, output, k)
tend=time.clock()
print(tend)

### change the coordinate system accordingly!!